# Dunders

I tipi primitivi, e.g., `int` hanno a disposizione diversi operatori, e.g., `+` o `==`, ma lo stesso non vale per le classi che definiamo noi. Python offre un insieme di metodi *dunder* che noi possiamo implementare per rendere i nostri tipi utilizzabili con gli operatori classici, e.g.,

- `+`, `-`, `*`, `/`
- `==`, `!=`
- `>=, <=, >, <`
- `~`
- `[key]`

Esempio: per creare un `set` di `Bacteria`, ogni batterio deve essere diverso dagli altri: come verificarlo? Abbiamo bisogno di sapere se, per due batteri `a`, `b`, `a == b`. Ma essendo di un tipo che abbiamo definito noi, Python non sa come come implemenetare `==`.

*[Reference online](https://docs.python.org/3/reference/datamodel.html)*

## `==`

In [ ]:
class Bacteria:
    def __init__(self, genome: str):
        self.genome = genome

    def __eq__(self, other):
        return isinstance(other, Bacteria) and self.genome == other.genome

genome = "ACGTGGTCA"
other_genome = "CGTCA"
a, b, c = Bacteria(genome), Bacteria(genome), Bacteria(other_genome)

a == b, a == c

(True, False)

## `+, -, *, /`
*[Reference online](https://docs.python.org/3/reference/datamodel.html#object.__add__)*

In [ ]:
class Genome:
    def __init__(self, genome: str):
        self.genome = genome

    def __add__(self, other):
        """Concatena questo e l'altro genoma."""
        return Genome(self.genome + other.genome)

genome = "ACGTGGTCA"
other_genome = "CGTCA"
genome_obj, other_genome_obj = Genome(genome), Genome(other_genome)

joined_genome = genome_obj + other_genome_obj
joined_genome.genome

'ACGTGGTCACGTCA'

## `~`
*[Reference online](https://docs.python.org/3/reference/datamodel.html#object.__invert__)*

In [ ]:
class Genome:
    def __init__(self, genome: str):
        self.genome = genome

    def complement(self, base: str) -> str:
        return "T" if base == "A" else\
            "A" if base == "T" else\
            "C" if base == "G" else\
            "G"


    def __invert__(self):
        """Complemento del genoma."""
        return Genome([self.complement(base) for base in self.genome])


genome = "ACGTGGTCA"
genome_obj = Genome(genome)
(~genome_obj).genome

['T', 'G', 'C', 'A', 'C', 'C', 'A', 'G', 'T']

## `[]`
*[Reference online](https://docs.python.org/3/reference/datamodel.html#object.__getitem__)*

In [ ]:
class Genome:
    def __init__(self, genome: str):
        self.genome = genome

    def __getitem__(self, key):
        return self.genome[key]


genome = "ACGTGGTCA"
genome_obj = Genome(genome)
genome_obj[3]

'T'

## `in`

In [ ]:
class Genome:
    def __init__(self, genome: str):
        self.genome = genome

    def __contains__(self, key):
        return key in self.genome


genome = "ACGTGGTCA"
genome_obj = Genome(genome)
"ACG" in genome_obj

True

---

# Inheritance

Come per gli organismi, le classi possono avere dei rapporti di ereditarietà, e.g., un `Dinosauro` è un `Arcosauro`, che a sua volta è un `Vertebrato`. Questi rapporti sono anche detti "di estensione". Una classe `B` (e quindi tutti gli oggetti di tipo `B`) che estende una classe `A`, eredita:
- tipo: se per un oggetto `b` di tipo `B` usiamo `isinstance(b, A)` avremo `True`
- metodi e campi: se `A` definisce un metodo `synthesize_hormone()`, allora anche in oggetti di tipo `B` potremo invocare `b.synthesize_hormone`

In fase di definizione classe, indichiamo:
```python
class Ereditante(Padre):
```

In [ ]:
class A:
    def greet(self):
        return "Hello!"

    def who(self):
        return f"{self.greet()} I am A"

class B(A):
    def who(self):
        # self.greet() viene ereditato da A!
        return f"{self.greet()} I am B"


a = A()
b = B()
print(a.who())
print(b.who())

Hello! I am A
Hello! I am B


## Inheritance e overload

Non dobbiamo necessariamente lasciarci definire dai nostri padri! Una classe che estende può sovrascrivere metodi e campi: questo fenomeno vien detto "overloading" in quanto stiamo dando diverse definizioni a uno stesso nome (il metodo/campo che stiamo sovrascrivendo).

In [ ]:
class C:
    def greet(self):
        return "I salute thee, fellow chap!"

    def who(self):
        return f"{self.greet()} I am C"

class D(A):
    def greet(self):
        return "Hi, buddy!"

    def who(self):
        # self.greet() viene ereditato da C, ma sovrascritto (overloaded)!
        return f"{self.greet()} I am D"


c = C()
d = D()
print(c.who())
print(d.who())

I salute thee, fellow chap! I am C
Hi, buddy! I am D


## Overloading... ma non troppo

Anche se facciamo overload, possiamo comunque accedere alle definizioni dei padri con `super`. `super` ci permette, e.g.,
- di invocare il costruttore del padre: `super()`
- di accedere a campi e metodi ereditati: `super.synthetisazion_start_codon`, `super.synthesize_atp()`

In un caso invocare metodi del padre è obbligatorio: nel costruttore. Quando definiamo il costruttore di una classe che eredita, dobbiamo invocare come prima istruzione il costruttore del padre: `super().__init__()`

In [ ]:
class Vertebrate:
    def __init__(self, has_vertebrae: bool = True):
        self.has_vertebrae = has_vertebrae

class Myxini(Vertebrate):
    def __init__(self):
        super().__init__(has_vertebrae = False)
        self.habitat = "deep sea"

class Lamprey(Vertebrate):
    def __init__(self):
        super().__init__(has_vertebrae = True)
        self.habitat = "deep sea"

hagfish = Myxini()
lamprey = Lamprey()

print(f"Does the hagfish have vertebrae? {hagfish.has_vertebrae}")
print(f"Does the lamprey have vertebrae? {lamprey.has_vertebrae}")

Does the hagfish have vertebrae? False
Does the lamprey have vertebrae? True


**Nota!** Non accediamo direttamente a *campi* del padre, ma possiamo accedere ai metodi della classe da cui stiamo ereditando:
- `super()`: usando `super().metodo(...)`, che funziona solo a un livello. Accediamo al padre, ma non agli antenati precedenti
- `Name.metodo(self, ...)`: usando `Name.metodo(self, ...)`, che funziona ad ogni livello.

La prima è preferibile per leggibilità del codice. In generale, se stiamo facendo overload a cascata che richiedono invocazioni all'indietro, fermiamoci a pensare se possiamo migliorare direttamente le classi.

In [ ]:
class Vertebrate:
    def __init__(self, has_vertebrae: bool = True):
        self.has_vertebrae = has_vertebrae

    def greet(self):
        return "Vertebrate"

class Myxini(Vertebrate):
    def __init__(self):
        super().__init__(has_vertebrae = False)
        self.habitat = "deep sea"

    def greet(self):
        return "Myxini"

class Lamprey(Vertebrate):
    def __init__(self):
        super().__init__(has_vertebrae = True)
        self.habitat = "deep sea"

    def greet(self):
        return "Lamprey"

class RiverLamprey(Lamprey):
    def __init__(self):
        super().__init__()
        self.habitat = "rivers"

    def greet(self):
        return f"RiverLamprey, child of {Lamprey.greet(self)}, grandchild of {Vertebrate.greet(self)}"

    def greet_with_super(self):
        return f"RiverLamprey, child of {super().greet()}"

river_lamprey = RiverLamprey()
print(river_lamprey.greet())
print(river_lamprey.greet_with_super())

RiverLamprey, child of Lamprey, grandchild of Vertebrate
RiverLamprey, child of Lamprey


---

# Up to you!

`str()` e' una funzione di casting che trasforma il dato oggetto in una stringa, se possibile. Il cast puo' avvenire se l'oggetto implementa un dunder specifico: quale? Cerca nella [documentazione](https://docs.python.org/3/reference/datamodel.html), e implementalo per la classe `Genome`.

2

In [ ]:
genome = "ACGTGGTCA"
genome_obj = Genome(genome)
str(genome_obj)

`in` e' un operatore che ci permette di dire se un oggetto e' in una collezione. Implementa una classe `GenomeBank` che definisce una collezione di genomi, e che in cui possiamo cercare un genoma specifico con `in`.

In [ ]:
import random


def random_genome() -> str:
    return random.choices("ACGT", k=25)


genomes = [random_genome()
           for _ in range(100)]
genome_objs = [Genome(genome)
               for genome in genomes]
bank = GenomeBank(genome_objs)

print("ACGTZZA" in bank)
print(all([obj in bank for obj in genome_objs]))